In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline
from collections import OrderedDict
from six.moves import cPickle

import os
import sys
import timeit

import scipy.io as sio
import numpy as np
import theano
import theano.tensor as T

import nnet as nn
import criteria as er
import util
import VAE
import SVAE
import update

In [2]:
def get_acc(pred, true):
    ll = pred - true
    ll = np.array(ll)
    acc = 1 - (np.nonzero(ll)[0].shape[0])/float(ll.shape[0])
    return acc

In [3]:
'''
train_file = 'timit_train_subset_1ok.npy'
valid_file = 'timit_valid_1ok.npy'
#test_file = 'test_1ok.npy'
    
train=np.load(train_file)
valid=np.load(valid_file)
#test=np.load(test_file)
'''



"\ntrain_file = 'timit_train_subset_1ok.npy'\nvalid_file = 'timit_valid_1ok.npy'\n#test_file = 'test_1ok.npy'\n    \ntrain=np.load(train_file)\nvalid=np.load(valid_file)\n#test=np.load(test_file)\n"

In [4]:
'''
print(np.where(train[2][1] == 1))
print(train[1][1])
'''

'\nprint(np.where(train[2][1] == 1))\nprint(train[1][1])\n'

In [5]:
'''
i=40
plt.figure(1)
plt.imshow(train[0][i].reshape(11,40).transpose(), extent=[0,11,0,40])
print train[1][i]
plt.show()
'''

'\ni=40\nplt.figure(1)\nplt.imshow(train[0][i].reshape(11,40).transpose(), extent=[0,11,0,40])\nprint train[1][i]\nplt.show()\n'

In [6]:
'''Load Data'''
train_file = 'timit_train_subset_1ok.npy'
valid_file = 'timit_valid_1ok.npy'
#test_file = 'test_1ok.npy'
    
train=np.load(train_file)
valid=np.load(valid_file)
#test=np.load(test_file)

    

print('Loading training set')    
train_feat, train_label = util.shared_dataset_timit(train)
print('Loading valid set')
valid_feat, valid_label = util.shared_dataset_timit(valid)
#test_feat, test_label = util.shared_dataset(test) 
    
  
'''Coefficient Initial'''        
batch_size = 300
n_epochs = 5
learning_rate = 0.1
    
n_train_batches = train_feat.get_value(borrow=True).shape[0] // batch_size
n_valid_batches = valid_feat.get_value(borrow=True).shape[0] // batch_size
#n_test_batches = test_feat.get_value(borrow=True).shape[0] // batch_size
print('number of minibatch at one epoch: train  %i, validation %i' %
    (n_train_batches, n_valid_batches))

Loading training set
Loading valid set
number of minibatch at one epoch: train  1191, validation 186


In [7]:
train_label.get_value(borrow=True)

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [8]:
z_dim = 10 #dimension of latent variable 
x_dim = train_feat.get_value(borrow=True).shape[1]
y_dim = train_label.get_value(borrow=True).shape[1]
activation = None
    
print(train_feat.get_value(borrow=True).shape[0])
print(train_label.get_value(borrow=True).shape[0])
print(train_feat.get_value(borrow=True).shape[1])
print(train_label.get_value(borrow=True).shape[1])

phi_1_struct=nn.NN_struct()
phi_1_struct.layer_dim = [x_dim+y_dim, 500, z_dim]
phi_1_struct.activation = [None, None]
    
theta_1_struct=nn.NN_struct()
theta_1_struct.layer_dim = [x_dim, 500, z_dim]
theta_1_struct.activation = [None, None]

theta_2_struct=nn.NN_struct()
theta_2_struct.layer_dim = [z_dim+x_dim, 500, y_dim]
theta_2_struct.activation = [None, None]

357376
357376
440
1984


In [9]:
index = T.lscalar()  # index to a [mini]batch
x = T.matrix('x')  # the data is presented as rasterized images
y = T.matrix('y')  # the labels are presented as signal vector   
rng = np.random.RandomState(1234)

#with open('model.save', 'rb') as f:
#    model = cPickle.load(f)


#model.input_x=x
#model.label_y=x
#model.phi_mu.OL.W.get_value()
#test[0][0]

In [10]:
index = T.lscalar()  # index to a [mini]batch
x = T.matrix('x')  # the data is presented as rasterized images
y = T.matrix('y')  # the labels are presented as signal vector   
rng = np.random.RandomState(1234)

#with open('model.save', 'rb') as f:
#    model = cPickle.load(f)


#model.input_x=x
#model.label_y=x
#model.phi_mu.OL.W.get_value()
#test[0][0]

In [11]:
classifier = SVAE.Supervised_VAE_v3_CE(
    rng=rng,
    input_x = x,
    label_y = y,
    batch_size = batch_size,
    phi_1_struct = phi_1_struct,
    theta_2_struct = theta_2_struct,
    in_dim = x_dim,
    out_dim = y_dim
    )

'''
cost = (classifier.cost)
        
gparams = [T.grad(cost, param) for param in classifier.params]
                   
updates = [
    (param, param - learning_rate * gparam)
    for param, gparam in zip(classifier.params, gparams)
]
'''


updates = update.adam(loss=classifier.cost, all_params=classifier.params, learning_rate=0.0005)


#classifier.theta_mu = model.phi_mu
#classifier.theta_sigma = model.phi_sigma

#print(classifier.theta_mu.OL.W.get_value())
#print(model.phi_mu.OL.W.get_value())
#print(parp)

In [12]:
print('... prepare training model')
train_model = theano.function(
    inputs=[index],
    outputs=[classifier.cost, classifier.predictor, classifier.label_y],
    updates=updates,
    givens={
        x: train_feat[index * batch_size : (index + 1) * batch_size, :],
        y: train_label[index * batch_size : (index + 1) * batch_size, :]
    }       
)   
    
print('... prepare validate model')
validate_model = theano.function(
    inputs=[index],
    outputs=classifier.cost,
    givens={
        x: valid_feat[index * batch_size : (index + 1) * batch_size, :],
        y: valid_label[index * batch_size : (index + 1) * batch_size, :]
    }        
)      

'''
print('... prepare test model')
test_model = theano.function(
    inputs=[index],
    outputs=[classifier.predictor, classifier.label_y],
    givens={
        x: test_feat[index * batch_size : (index + 1) * batch_size, :],
        y: test_label[index * batch_size : (index + 1) * batch_size, :]
    }        
)
'''

... prepare training model
... prepare validate model


"\nprint('... prepare test model')\ntest_model = theano.function(\n    inputs=[index],\n    outputs=[classifier.predictor, classifier.label_y],\n    givens={\n        x: test_feat[index * batch_size : (index + 1) * batch_size, :],\n        y: test_label[index * batch_size : (index + 1) * batch_size, :]\n    }        \n)\n"

In [13]:
###############
# TRAIN MODEL #
###############
'''
Define :
    xx_loss : Cost function value
    xx_score : Classification accuracy rate
'''        
    
print('... training')
    
# early-stopping parameters
patience = 10000  # look as this many examples regardless
patience_increase = 2  # wait this much longer when a new best is
                           # found
improvement_threshold = 0.995  # a relative improvement of this much is
                                   # considered significant
validation_frequency = min(n_train_batches, patience // 2)
                                  # go through this many
                                  # minibatche before checking the network
                                  # on the validation set; in this case we
                                  # check every epoch
    
#validation_frequency = n_train_batches
    
best_iter = 0
best_train_loss = np.inf
best_validation_loss = np.inf  
test_loss = np.inf
train_score = 0.
validation_score = 0.
test_score = 0.    
start_time = timeit.default_timer()

epoch = 0
done_looping = False

n_epochs = 5


while (epoch < n_epochs) and (not done_looping):
    epoch = epoch + 1
    train_acc=[]
    for minibatch_index in range(n_train_batches):

        [minibatch_avg_cost, pred, lab] \
        = train_model(minibatch_index)
        #print(minibatch_index)
        #print(pred)               
        # iteration number
        #print('minibatch %i/%i' % (minibatch_index, n_train_batches))
        
        iter = (epoch - 1) * n_train_batches + minibatch_index
        if iter % 100 == 0:
            print('minibatch %i/%i' % (minibatch_index, n_train_batches))
            
        train_acc.append(get_acc(pred, np.nonzero(lab)[1]))
            
        if (iter + 1) % validation_frequency == 0:
            # compute loss on validation set
            validation_losses = [validate_model(i) for i in range(n_valid_batches)] 
            this_validation_loss = np.mean(validation_losses)
                
            #print('CE loss: %f' % (np.mean(KL_loss)))
            #print('encoder mu: %f, sigma: %f' % (ec_m, ec_s))
            #print('decoder mu: %f, sigma: %f' % (dc_m, dc_s))
            #print(ec_m)
            #print(dc_m)
            #print(ec_s)
            #print(dc_s)
            
            print('epoch training accuracy: %f, training loss: %f' \
                % (np.mean(np.array(train_acc)), np.mean(minibatch_avg_cost)))
            print(
                'epoch %i, minibatch %i/%i, validation loss %f' %
                (
                    epoch,
                    minibatch_index + 1,
                    n_train_batches,
                    this_validation_loss
                )
            )

            # if we got the best validation score until now
            if this_validation_loss < best_validation_loss:
                #improve patience if loss improvement is good enough
                if (
                    this_validation_loss < best_validation_loss *
                    improvement_threshold
                ):
                    patience = max(patience, iter * patience_increase)

                best_validation_loss = this_validation_loss   
                best_iter = iter
                '''
                # get training accuracy
                print('best training accuracy: %f' % (np.mean(np.array(train_acc))))
                # test it on the test set
                #test_losses = [test_model(i) for i in range(n_test_batches)]
                #test_score = np.mean(test_losses)

                print(('epoch %i, minibatch %i/%i, best train accuracy: %f') % \
                        (epoch, minibatch_index + 1, n_train_batches, \
                        np.mean(np.array(train_acc))))
                '''
                
                '''
                test_acc=[]
                for minibatch_index in range(n_test_batches):
                    [pred_test, lab_test]= test_model(minibatch_index)
                    #print(pred)
                    #print(np.nonzero(lab)[1])
                    # iteration number
                    iter = minibatch_index
                    test_acc.append(get_acc(pred_test, np.nonzero(lab_test)[1]))
    
                print('test accuracy: %f' % (np.mean(test_acc)))
                '''
                

        if patience <= iter:
            done_loop
            ing = True
            break

... training
minibatch 0/1191
minibatch 100/1191
minibatch 200/1191
minibatch 300/1191
minibatch 400/1191
minibatch 500/1191
minibatch 600/1191
minibatch 700/1191
minibatch 800/1191
minibatch 900/1191
minibatch 1000/1191
minibatch 1100/1191
epoch training accuracy: 0.308805, training loss: 1.291277
epoch 1, minibatch 1191/1191, validation loss 1.470716
minibatch 9/1191
minibatch 109/1191
minibatch 209/1191
minibatch 309/1191
minibatch 409/1191
minibatch 509/1191
minibatch 609/1191
minibatch 709/1191
minibatch 809/1191
minibatch 909/1191
minibatch 1009/1191
minibatch 1109/1191
epoch training accuracy: 0.894545, training loss: 0.088603
epoch 2, minibatch 1191/1191, validation loss 0.166461
minibatch 18/1191
minibatch 118/1191
minibatch 218/1191
minibatch 318/1191
minibatch 418/1191
minibatch 518/1191
minibatch 618/1191
minibatch 718/1191
minibatch 818/1191
minibatch 918/1191
minibatch 1018/1191
minibatch 1118/1191
epoch training accuracy: 0.990912, training loss: 0.017033
epoch 3, miniba

In [14]:
print('... prepare test model')
test_model = theano.function(
    inputs=[index],
    outputs=[classifier.predictor, classifier.label_y, classifier.PR_y_pred],
    givens={
        x: valid_feat[index * batch_size : (index + 1) * batch_size, :],
        y: valid_label[index * batch_size : (index + 1) * batch_size, :]
    }        
)

... prepare test model


In [15]:
#validation_losses = [validate_model(i) for i in range(n_valid_batches)]
[pred, y, pr_y_pred] = test_model(0)

In [25]:
classifier.phi_mu.HL_1.W.get_value()

array([[-0.08598345,  0.04936071, -0.00484833, ...,  0.11585402,
        -0.12958887,  0.05190004],
       [ 0.16762261,  0.0967848 ,  0.05059614, ..., -0.12501037,
         0.01127056,  0.0035666 ],
       [-0.00532561,  0.17155547,  0.04719282, ..., -0.11836273,
        -0.04804441,  0.03040252],
       ..., 
       [-0.19622377,  0.1814644 ,  0.21150714, ..., -0.13161674,
         0.0925743 , -0.00544427],
       [-0.04907462, -0.03964802, -0.18481621, ..., -0.34770074,
         0.41454947, -0.09753846],
       [-0.57451648,  0.66295689, -0.2089753 , ..., -0.63351285,
         0.37848681, -0.50703043]], dtype=float32)

In [ ]:
with open('timit_model_step_ce_1l_500h_z10.save', 'wb') as f:
    cPickle.dump(classifier, f, protocol=cPickle.HIGHEST_PROTOCOL)

In [ ]:
aaa=classifier.phi_mu.HL_1.W.get_value()